# Temporal Regularized Matrix Factorization

The following Python codes reproduce the paper:

> H.-F. Yu, N. Rao, and I. S. Dhillon. Temporal regularized matrixfactorization for high-dimensional time series prediction. Advances in neural information processing systems, vol. 29, pp. 847–855, 2016.

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def generate_Psi(T, d):
    Psi = []
    for k in range(0, d + 1):
        if k == 0:
            Psi.append(np.append(np.zeros((T - d, d)), np.eye(T - d), axis = 1))
        else:
            Psi.append(np.append(np.append(np.zeros((T - d, d - k)), np.eye(T - d), axis = 1), 
                                 np.zeros((T - d, k)), axis = 1))
    return Psi

def update_cg(w, r, q, Aq, rold):
    alpha = rold / np.inner(q, Aq)
    w = w + alpha * q
    r = r - alpha * Aq
    rnew = np.inner(r, r)
    q = r + (rnew / rold) * q
    return w, r, q, rnew

def ell_w(ind, W, X, rho):
    return X @ ((W.T @ X) * ind).T + rho * W

def conj_grad_w(sparse_mat, ind, W, X, rho, maxiter = 5):
    rank, dim1 = W.shape
    w = np.reshape(W, -1, order = 'F')
    r = np.reshape(X @ sparse_mat.T - ell_w(ind, W, X, rho), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim1), order = 'F')
        Aq = np.reshape(ell_w(ind, Q, X, rho), -1, order = 'F')
        w, r, q, rold = update_cg(w, r, q, Aq, rold)
    return np.reshape(w, (rank, dim1), order = 'F')

def ell_x(sparse_mat, ind, W, X, A, Psi, d, rho, lambda0):
    dim1, dim2 = sparse_mat.shape
    rank = W.shape[0]
    temp = np.zeros((d * rank, dim2 - d))
    for k in range(1, d + 1):
        temp[(k - 1) * rank : k * rank, :] = X[:, d - k : dim2 - k]
    temp0 = X @ Psi[0].T - A @ temp
    temp1 = np.zeros((rank, dim2))
    for k in range(d):
        temp1 += A[:, k * rank : (k + 1) * rank].T @ temp0 @ Psi[k + 1]
    return W @ ((W.T @ X) * ind) + rho * X + lambda0 * (temp0 @ Psi[0] - temp1)

def conj_grad_x(sparse_mat, ind, W, X, A, Psi, d, rho, lambda0, maxiter = 5):
    rank, dim1 = W.shape
    dim2 = X.shape[1]
    x = np.reshape(X, -1, order = 'F')
    r = np.reshape(W @ sparse_mat - ell_x(sparse_mat, ind, W, X, A, Psi, d, rho, lambda0), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim2), order = 'F')
        Aq = np.reshape(ell_x(sparse_mat, ind, W, Q, A, Psi, d, rho, lambda0), -1, order = 'F')
        x, r, q, rold = update_cg(x, r, q, Aq, rold)
    return np.reshape(x, (rank, dim2), order = 'F')

def notmf(dense_mat, sparse_mat, rank, d, lambda0, rho, maxiter):
    dim1, dim2 = sparse_mat.shape
    W = 0.01 * np.random.randn(rank, dim1)
    X = 0.01 * np.random.randn(rank, dim2)
    A = 0.01 * np.random.randn(rank, d)
    if np.isnan(sparse_mat).any() == False:
        ind = sparse_mat != 0
        pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    elif np.isnan(sparse_mat).any() == True:
        pos_test = np.where((dense_mat != 0) & (np.isnan(sparse_mat)))
        ind = ~np.isnan(sparse_mat)
        sparse_mat[np.isnan(sparse_mat)] = 0
    dense_test = dense_mat[pos_test]
    del dense_mat
    Psi = generate_Psi(dim2, d)
    indx = np.zeros((d, dim2 - d), dtype = np.int_)
    for k in range(d):
        indx[k, :] = np.arange(d - k, dim2 - k)
    show_iter = 100
    for it in range(maxiter):
        W = conj_grad_w(sparse_mat, ind, W, X, rho)
        A_mat = np.zeros((rank, d * rank))
        for k in range(d):
            A_mat[:, k * rank : (k + 1) * rank] = np.diag(A[:, k])
        X = conj_grad_x(sparse_mat, ind, W, X, A_mat, Psi, d, rho, lambda0)
        for r in range(rank):
            A[r, :] = np.linalg.lstsq(X[r, indx].T, X[r, d :], rcond = None)[0]
        mat_hat = W.T @ X
        if (it + 1) % show_iter == 0:
            temp_hat = mat_hat[pos_test]
            print('Iter: {}'.format(it + 1))
            print('Loss function: {}'.format(np.linalg.norm((sparse_mat - mat_hat) * ind, 'fro')))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            print()
    return mat_hat, W, X, A

def notmf_dic(obs, W, X, A, d, lambda0, rho):
    dim1, dim2 = obs.shape
    rank = X.shape[0]
    if np.isnan(obs).any() == False:
        ind = obs != 0
    elif np.isnan(obs).any() == True:
        ind = ~np.isnan(obs)
        obs[np.isnan(obs)] = 0
    Psi = generate_Psi(dim2, d)
    indx = np.zeros((d, dim2 - d), dtype = np.int_)
    for k in range(d):
        indx[k, :] = np.arange(d - k, dim2 - k)
    A_mat = np.zeros((rank, d * rank))
    for k in range(d):
        A_mat[:, k * rank : (k + 1) * rank] = np.diag(A[:, k])
    X = conj_grad_x(obs, ind, W, X, A_mat, Psi, d, rho, lambda0)
    for r in range(rank):
        A[r, :] = np.linalg.lstsq(X[r, indx].T, X[r, d :], rcond = None)[0]
    return X, A

def var4cast(X, A, d, delta):
    dim1, dim2 = X.shape
    X_hat = np.append(X, np.zeros((dim1, delta)), axis = 1)
    A_mat = np.zeros((rank, d * rank))
    for k in range(d):
        A_mat[:, k * rank : (k + 1) * rank] = np.diag(A[:, k])
    for t in range(delta):
        X_hat[:, dim2 + t] = A_mat @ X_hat[:, dim2 + t - np.arange(1, d + 1)].T.reshape(dim1 * d)
    return X_hat

from ipywidgets import IntProgress
from IPython.display import display

def rolling4cast(dense_mat, sparse_mat, pred_step, delta, rank, d, lambda0, rho, maxiter):
    dim1, T = sparse_mat.shape
    start_time = T - pred_step
    max_count = int(np.ceil(pred_step / delta))
    mat_hat = np.zeros((dim1, max_count * delta))
    f = IntProgress(min = 0, max = max_count) # instantiate the bar
    display(f) # display the bar
    for t in range(max_count):
        if t == 0:
            _, W, X, A = notmf(dense_mat[:, : start_time], sparse_mat[:, : start_time], 
                                 rank, d, lambda0, rho, maxiter)
        else:
            X, A = notmf_dic(sparse_mat[:, : start_time + t * delta], W, X_new, A, d, lambda0, rho)
        X_new = var4cast(X, A, d, delta)
        mat_hat[:, t * delta : (t + 1) * delta] = W.T @ X_new[:, - delta :]
        f.value = t
    small_dense_mat = dense_mat[:, start_time : T]
    pos = np.where(small_dense_mat != 0)
    mape = compute_mape(small_dense_mat[pos], mat_hat[pos])
    rmse = compute_rmse(small_dense_mat[pos], mat_hat[pos])
    print('Prediction MAPE: {:.6}'.format(mape))
    print('Prediction RMSE: {:.6}'.format(rmse))
    print()
    return mat_hat, W, X, A

In [2]:
import numpy as np

dense_mat = np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 4):
    dense_mat = np.append(dense_mat, np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)

In [3]:
import time
start = time.time()
dim1, dim2 = dense_mat.shape
rank = 10
pred_step = 7 * 24
delta = 6
d = 12
lambda0 = 1
rho =  5
maxiter = 50
mat_hat, W, X, A = rolling4cast(dense_mat[:, : 24 * 7 * 10], dense_mat[:, : 24 * 7 * 10], 
                                pred_step, delta, rank, d, lambda0, rho, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

IntProgress(value=0, max=28)

Prediction MAPE: 0.219009
Prediction RMSE: 4.77648

Running time: 948 seconds


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>